In [33]:
import os
log_dir = '/vol/vssp/mammo2/will/logs/new'
fCount = 0
while os.path.exists(os.path.join(log_dir, 'training', str(fCount))):
    fCount+=1
    
print(os.path.exists(os.path.join(log_dir, 'training')))
print(log_dir + 'hello')

True
/vol/vssp/mammo2/will/logs/newhello


In [1]:
# fold
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard
from time import time
import os
import tensorflow as tf
from keras import applications

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/vol/vssp/mammo2/will/logs/new', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        fCount=0
        while os.path.exists(os.path.join(log_dir, 'training' + '_' + str(fCount))):
            fCount+=1
        training_log_dir = os.path.join(log_dir, 'training' + '_' + str(fCount))
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation' + '_' + str(fCount))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x

def fourCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    #model.add(Conv2D(32, (3,3), activation='relu'))   
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model
    
def fiveCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model
    
    
def main():
    import keras
    print('keras version: ', keras.__version__)
    print('TensorFlow version: ', tf.__version__)    
    valStats = []
    folds = 16
    normalise = 0
    for crossVal in range(folds):
        print('\nLoad images...')

        # Get images and labels
        data = {'img': 0, 'label': 0}
        dataSpecs = {'classLength': []}
        dataSpecs['classLength'] = []
        data['img'] = np.concatenate((
                #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/0/*', dataSpecs), # Class 0
                #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/1/*', dataSpecs) # Class 1
    #             get_images('/user/HS204/wm0015/student/allCalcs/0/*', dataSpecs), # Class 0
    #             get_images('/user/HS204/wm0015/student/allCalcs/1/*', dataSpecs) # Class 1
    #             get_images('/vol/vssp/mammo2/will/data/camelyon/combine/normal/*', dataSpecs), # Class 0
    #             get_images('/vol/vssp/mammo2/will/data/camelyon/combine/tumour/*', dataSpecs) # Class 1
                get_images('/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/normal/*', dataSpecs), # Class 0
                get_images('/vol/vssp/cvpwrkspc01/scratch/wm0015/crops256/malignant/*', dataSpecs) # Class 1
        ))      

        # Normalise
        print('Normalising by: ', 2**normalise)
        data['img'] = data['img']/(2**normalise)  
        normalise+= 1
        print(data['img'].dtype)
        # Print image    
        img_calc = data['img']   
        plt.imshow(img_calc[0], cmap='gray')
        plt.show()

        # Create one hot labels
        labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
        labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
        data['label'] = np.concatenate((
                get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
                get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
        ))
        # Drop from 3 colour channels to 1 (greyscale)
        if 1==0:
            data['img'] = data['img'][:,:,:,0]
            data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
            print('new data shape = ', data['img'].shape)


        # Shuffle data
        seed = 33
    #     np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0])    
        data['img'] = data['img'][shuffleMask, :, :, :]
        data['label'] = data['label'][shuffleMask, :]

        # Split traing and validation data        
        splitRatio = 0.9
        splitPoint = math.floor(data['img'].shape[0]*splitRatio)
        train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
        val_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
#         print('train_data[img].shape = ', train_data['img'].shape)
#         print('train_data[label].shape = ', train_data['label'].shape)
#         print('val_data[img].shape = ', val_data['img'].shape)
#         print('valdiation_data[label].shape = ', val_data['label'].shape)
#         print('dataSpecs = ', dataSpecs['classLength'])

        print('Compute bottleneck features...')
        vggModel = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (256, 256, 3))
        # Freeze all layers
        for layer in vggModel.layers:
            layer.trainable = False
        # Add custom final layer
        bNModel = vggModel.output
        bNModel = Flatten()(bNModel)
        final_model = keras.models.Model(inputs=vggModel.input, outputs=bNModel)
        train_bNFeatures = {'img': 0, 'label': train_data['label']}
        val_bNFeatures = {'img': 0, 'label': val_data['label']}
        train_bNFeatures['img'] = final_model.predict(train_data['img'], batch_size=16)
        val_bNFeatures['img'] = final_model.predict(val_data['img'], batch_size=16)
        print('train_bNFeatures[img].shape = ', train_bNFeatures['img'].shape)
        print('val_bNFeatures[img].shape = ', val_bNFeatures['img'].shape)
        
        print('Train head...')
        head = Sequential()
        #head.add(Dense(32, input_dim=train_bNFeatures['img'].shape[1], activation='relu'))
        head.add(Dense(2, activation='softmax'))
        
        sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
        head.compile(loss='binary_crossentropy',
                    optimizer=adam,
                     metrics=['accuracy'])
        tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()), write_images=True)
        if folds == 1:
            head.summary()  

        # Train
        config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': 30} )
        sess = tf.Session(config=config) 
        keras.backend.set_session(sess)
        head.fit(train_bNFeatures['img'], train_data['label'], 
                batch_size=100, epochs=200, verbose=1,
                callbacks=[TrainValTensorBoard(write_graph=False)],
                validation_data=(val_bNFeatures['img'], val_bNFeatures['label']))

        # Evaluate
        score = head.evaluate(val_bNFeatures['img'], val_bNFeatures['label'], verbose=0)
        valStats.append(score)
        print('The score is......\n', score)
        
#         model_final = keras.Model(input = model.input, output = customVgg)
        

# #         model.compile(loss='binary_crossentropy',
# #                 optimizer=adam,
# #                 metrics=['accuracy']) 
  

    valStats = np.asarray(valStats)
    print('Validations: \n', valStats[:, 1])
    print('Average loss: ', sum(valStats[:, 0])/folds)
    print('Average validation: ', sum(valStats[:, 1])/folds)
    
if __name__ == "__main__":
    main()

Using TensorFlow backend.


keras version:  2.2.0
TensorFlow version:  1.7.1

Load images...
Normalising by:  1


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


float64


<Figure size 640x480 with 1 Axes>

...

<Figure size 640x480 with 1 Axes>

Compute bottleneck features...
train_bNFeatures[img].shape =  (2673, 32768)
val_bNFeatures[img].shape =  (297, 32768)
Train head...
Train on 2673 samples, validate on 297 samples
Epoch 1/200
2673/2673 [==============================] - 1s 367us/step - loss: 1.2241 - acc: 0.4987 - val_loss: 0.6946 - val_acc: 0.4882
Epoch 2/200
2673/2673 [==============================] - 0s 140us/step - loss: 0.7597 - acc: 0.4994 - val_loss: 0.7573 - val_acc: 0.5118
Epoch 3/200
2673/2673 [==============================] - 1s 225us/step - loss: 0.7237 - acc: 0.5043 - val_loss: 0.7257 - val_acc: 0.5118
Epoch 4/200
2673/2673 [==============================] - 0s 163us/step - loss: 0.7132 - acc: 0.5047 - val_loss: 0.7410 - val_acc: 0.5118
Epoch 5/200
2673/2673 [==============================] - 0s 131us/step - loss: 0.7050 - acc: 0.5099 - val_loss: 0.7092 - val_acc: 0.5118
Epoch 6/200
2673/2673 [==============================] - 0s 117us/step - loss: 0.7151 - acc: 0.5054 - val_loss: 0.7200 - val_acc: 0.5118

2673/2673 [==============================] - 0s 113us/step - loss: 0.7098 - acc: 0.5181 - val_loss: 0.6935 - val_acc: 0.5118
Epoch 60/200
2673/2673 [==============================] - 0s 123us/step - loss: 0.7396 - acc: 0.5159 - val_loss: 0.7419 - val_acc: 0.5118
Epoch 61/200
2673/2673 [==============================] - 0s 149us/step - loss: 0.7289 - acc: 0.4875 - val_loss: 0.7181 - val_acc: 0.4882
Epoch 62/200
2673/2673 [==============================] - 0s 133us/step - loss: 0.7204 - acc: 0.5036 - val_loss: 0.7269 - val_acc: 0.4882
Epoch 63/200
2673/2673 [==============================] - 0s 142us/step - loss: 0.7863 - acc: 0.4931 - val_loss: 0.7781 - val_acc: 0.4882
Epoch 64/200
2673/2673 [==============================] - 0s 130us/step - loss: 0.7641 - acc: 0.4938 - val_loss: 0.8734 - val_acc: 0.5118
Epoch 65/200
2673/2673 [==============================] - 0s 118us/step - loss: 0.7362 - acc: 0.4972 - val_loss: 0.6929 - val_acc: 0.5118
Epoch 66/200
2673/2673 [=======================

2673/2673 [==============================] - 0s 121us/step - loss: 0.9156 - acc: 0.4848 - val_loss: 0.7395 - val_acc: 0.5118
Epoch 119/200
2673/2673 [==============================] - 0s 114us/step - loss: 0.8166 - acc: 0.5155 - val_loss: 0.8628 - val_acc: 0.4882
Epoch 120/200
2673/2673 [==============================] - 0s 151us/step - loss: 0.7317 - acc: 0.5062 - val_loss: 0.7080 - val_acc: 0.5118
Epoch 121/200
2673/2673 [==============================] - 0s 130us/step - loss: 0.7373 - acc: 0.4972 - val_loss: 0.7062 - val_acc: 0.5118
Epoch 122/200
2673/2673 [==============================] - 0s 123us/step - loss: 0.7406 - acc: 0.4957 - val_loss: 0.8596 - val_acc: 0.5118
Epoch 123/200
2673/2673 [==============================] - 0s 128us/step - loss: 0.7434 - acc: 0.4949 - val_loss: 0.7485 - val_acc: 0.5118
Epoch 124/200
2673/2673 [==============================] - 0s 130us/step - loss: 0.7124 - acc: 0.5140 - val_loss: 0.7005 - val_acc: 0.4882
Epoch 125/200
2673/2673 [================

2673/2673 [==============================] - 0s 122us/step - loss: 0.7092 - acc: 0.5043 - val_loss: 0.6997 - val_acc: 0.4882
Epoch 178/200
2673/2673 [==============================] - 0s 163us/step - loss: 0.7566 - acc: 0.5103 - val_loss: 0.8518 - val_acc: 0.5118
Epoch 179/200
2673/2673 [==============================] - 0s 130us/step - loss: 0.7222 - acc: 0.4983 - val_loss: 0.8141 - val_acc: 0.5118
Epoch 180/200
2673/2673 [==============================] - 0s 139us/step - loss: 0.8224 - acc: 0.4931 - val_loss: 1.0866 - val_acc: 0.5118
Epoch 181/200
2673/2673 [==============================] - 0s 131us/step - loss: 0.8057 - acc: 0.5024 - val_loss: 0.7068 - val_acc: 0.5118
Epoch 182/200
2673/2673 [==============================] - 0s 118us/step - loss: 0.7265 - acc: 0.5125 - val_loss: 0.8107 - val_acc: 0.5118
Epoch 183/200
2673/2673 [==============================] - 0s 120us/step - loss: 0.7386 - acc: 0.5208 - val_loss: 0.6931 - val_acc: 0.5118
Epoch 184/200
2673/2673 [================

In [7]:
print('hello world')

hello world


In [12]:
x = [[0.6940992025681484, 0.3950617283950617], [0.6884314866713536, 0.5185185185185185], [0.6855149121932042, 0.5679012345679012]]
print(x[0])
print(sum(x[:][1])/2)

[0.6940992025681484, 0.3950617283950617]
0.603475002594936


In [35]:
x = [[6.5308382717179665, 0.5925925925925926], [0.16470135086112553, 0.9444444444444444], [0.15422181794304907, 0.9444444444444444], [0.19875195217721256, 0.9382716049382716]] 
x = np.asarray(x)
print(x, '\n')
print(x[0][0])
print(x[:, 0])
print(x.shape)

[[6.53083827 0.59259259]
 [0.16470135 0.94444444]
 [0.15422182 0.94444444]
 [0.19875195 0.9382716 ]] 

6.5308382717179665
[6.53083827 0.16470135 0.15422182 0.19875195]
(4, 2)
